<a href="https://colab.research.google.com/github/passionlee428/my-repo/blob/master/Alex_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

!nvidia-smi
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim 
import torch.functional as F
print(torch.__version__)

Mon Apr 27 07:53:39 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
# Number of classes
num_classes = 10

# Loss function & Optimizers
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR

# Hyperparameters
num_epochs = 300
batch_size = 128
learning_rate = 0.001
weight_decay = 1e-4
lr_min = 0.1 * learning_rate


class MyAlexNet(nn.Module):
    def __init__(self):
        super(MyAlexNet, self).__init__()
        
        self.features = nn.Sequential(
                                      nn.Conv2d(3, 96, 3, stride=2, padding=1),
                                      nn.ReLU(),
                                      nn.MaxPool2d(3, stride=2),
                                      nn.Conv2d(96, 256, 3, padding=1),
                                      nn.ReLU(),
                                      nn.MaxPool2d(3, stride=2),
                                      nn.Conv2d(256, 384, 3, padding=1),
                                      nn.ReLU(),
                                      nn.Conv2d(384, 384, 3, padding=1),
                                      nn.ReLU(),
                                      nn.Conv2d(384, 384, 3, padding=1),
                                      nn.ReLU(),
                                      nn.MaxPool2d(3, stride=2),
        )
        self.classifier = nn.Sequential(
                                        nn.Flatten(),
                                        nn.Dropout(),
                                        nn.Linear(384, 4096),
                                        nn.ReLU(),
                                        nn.Dropout(),
                                        nn.Linear(4096, 4096),
                                        nn.ReLU(),
                                        nn.Linear(4096, num_classes)
        )

    
    def forward(self, inputs):
        
        hidden = self.features(inputs)
        outputs = self.classifier(hidden)
        
        return outputs

# Prepare optimizers
model = MyAlexNet().cuda()
optimizer = optimizer(model.parameters(), 
                             lr=learning_rate, 
                             weight_decay=weight_decay)

In [0]:
import sys
sys.path.insert(0, "..")
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms

# CIFAR preprocessing 
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# DataLoader for CIFAR-10
cifar_train = torchvision.datasets.CIFAR10(root="./", 
                                         train=True, 
                                         download=True,
                                         transform=transform_train
                                         )
cifar_test = torchvision.datasets.CIFAR10(root="./", 
                                        train=False, 
                                        download=True,
                                        transform=transform_test)

train_loader = DataLoader(cifar_train, 
                          batch_size=batch_size, 
                          shuffle=True, 
                          num_workers=4)

test_loader = DataLoader(cifar_test, 
                         batch_size=batch_size, 
                         shuffle=False, 
                         num_workers=4)

Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


In [0]:
accuracy_graph = {'train':[], 'test':[], 'epoch': []}
loss_graph = {'train':[], 'test':[], 'epoch': []}

class accumulator():
    def __init__(self):
        self.cnt = 0
        self.obs = 0
        self.loss = 0

    def add(self, predict, output):

        assert len(predict) == len(output)

        self.cnt += self.count(predict, output).cpu().numpy()
        self.obs += len(predict)
    
    def count(self, predict, output):
        correct = torch.sum(torch.argmax(predict, dim=1) == output)
        
        return correct
        
    def loss_update(self, loss, batch_size):
        self.loss += loss * batch_size

    def running_stat(self):
        if self.obs != 0:
            run_acc = 100 * self.cnt / self.obs
            run_loss = self.loss / self.obs
        else: 
            run_acc = 0
            rum_loss = 0

        return run_acc, run_loss

    def reset(self):
        self.__init__()

In [0]:
# Template for running epoch
def run_epoch(data_loader, metric_accumulator, train=False):

    if train:
        model.train()
        scheduler = lr_scheduler(optimizer, 
                              len(data_loader), 
                              eta_min=lr_min)
    else:
        model.eval()
    
    for inputs, targets in data_loader:

        # Upload data to GPU
        inputs, targets = inputs.cuda(), targets.cuda()
        if train:
            predict = model(inputs)    
        else:
            with torch.autograd.no_grad():
                predict = model(inputs)

        # Compute loss
        loss = criterion(predict, targets)

        # Backpropagation
        if train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()   
            scheduler.step()

        metric_accumulator.add(predict, targets)
        metric_accumulator.loss_update(loss, batch_size)

# Log initiate
with open("log_alexnet.txt", "w") as log:
    init_log = "Epoch: {}, Batch size: {}, Optimizer: {}".format(num_epochs, 
                                                                 batch_size, 
                                                                 optimizer)
    log.write(init_log)

# Experiment
for epoch in range(num_epochs):

    train_accuracy = accumulator()
    test_accuracy = accumulator()

    # Training
    run_epoch(train_loader, train_accuracy, train=True)

    # Evaluation
    if epoch % 10 ==0:
            
        run_epoch(test_loader, test_accuracy, train=False)

        train_acc, train_loss = train_accuracy.running_stat()
        test_acc, test_loss = test_accuracy.running_stat()
        accuracy_graph['train'].append(train_acc)
        accuracy_graph['test'].append(test_acc) 
        accuracy_graph['epoch'].append(epoch)
        loss_graph['train'].append(train_loss)
        loss_graph['test'].append(test_loss)
        loss_graph['epoch'].append(epoch)
        
        # Upodate log
        log_accuracy = "\ntrain accuracy: {:.2f}%, test accuracy: {:.2f}%".format(train_acc, 
                                                                                  test_acc)
        log_epoch = "\nepoch:{}, train_loss: {:.4f}, test_loss: {:.4f}".format(epoch+1, 
                                                                               train_loss, 
                                                                               test_loss)

        with open("log_alexnet.txt", "a") as log:
            log.write(log_accuracy)
            log.write(log_epoch)
            print(log_accuracy, log_epoch) 

        train_accuracy.reset(), test_accuracy.reset()
        
        # Model save
        torch.save(model.state_dict(), "./alexnet.pt")


train accuracy: 27.57%, test accuracy: 37.58% 
epoch:1, train_loss: 1.8563, test_loss: 1.6220

train accuracy: 60.99%, test accuracy: 65.11% 
epoch:11, train_loss: 1.1153, test_loss: 0.9991

train accuracy: 67.48%, test accuracy: 71.13% 
epoch:21, train_loss: 0.9468, test_loss: 0.8387

train accuracy: 70.66%, test accuracy: 73.63% 
epoch:31, train_loss: 0.8591, test_loss: 0.7656

train accuracy: 72.32%, test accuracy: 74.77% 
epoch:41, train_loss: 0.8102, test_loss: 0.7262

train accuracy: 73.67%, test accuracy: 76.19% 
epoch:51, train_loss: 0.7600, test_loss: 0.7040
